# Dropping a New Ball

In [ ]:
from IPython.display import Image
im="https://www.thesun.co.uk/wp-content/uploads/2025/07/hilarious-moment-trumps-golf-caddy-1012803955.jpg?strip=all&w=634"
Image(url=im, width=600, height=600)

In [ ]:
#https://www.kaggle.com/code/cheeseexports/big-zippa
from zlib import compress

def zip_src(src):
 compression_level = 9 # Max Compression

 # We prefer that compressed source not end in a quotation mark
 while (compressed := compress(src, compression_level))[-1] == ord('"'): src += b"#"

 def sanitize(b_in):
  """Clean up problematic bytes in compressed b-string"""
  b_out = bytearray()
  for b in b_in:
   if   b==0:         b_out += b"\\x00"
   elif b==ord("\r"): b_out += b"\\r"
   elif b==ord("\\"): b_out += b"\\\\"
   else: b_out.append(b)
  return b"" + b_out

 compressed = sanitize(compressed)

 delim = b'"""' if ord("\n") in compressed or ord('"') in compressed else b'"'

 return b"#coding:L1\nimport zlib\nexec(zlib.decompress(bytes(" + \
  delim + compressed + delim + \
  b',"L1")))'

In [ ]:
import zipfile, json, os, copy
from zipfile import ZipFile
import re, gc, os, sys, zlib, base64
from collections import Counter
from nltk.util import ngrams
import py_compile

#Add hashing for input for more compression

p="/kaggle/input/google-code-golf-2025/code_golf_utils"
sys.path.append(p)
from code_golf_utils import *

#Since data is fized to [0,9] you can build your own compression algo
#This is just for illustration purposes
def theBirdie(s,a):
    c = Counter(list(ngrams(s, 2)))
    c=''.join(c.most_common(1)[0][0])
    s = s.replace(c,a)
    return s, c, a

def theEagle(s):
    m=[]
    alphas='abcdefghijklmnopqrstuvwxyzABCDEFGHJKLMNPQRSTUVWXYZ'
    s=str(s)
    s=s.replace(' ','')
    s=s.replace("'",'"')
    s=s.replace('output','O')
    s=s.replace('input','I')
    l=len(s)
    for a in alphas: 
        s, m1, m2 = theBirdie(s,a)
        if len(s)<l-5: #early stop
            m.append([m1,m2])
            l=len(s)
        else: 
            m.append([m1,m2]) #have to do final one since already replaced
            break
    m = m[::-1]
    m = ''.join(''.join(k) for k in m)
    return s,m

def theScorecard(s,m):
    sc = """def p(g):
 d='""" + str(s) + """'
 m='""" + str(m) + """'
 m=[[m[i:i+2],m[i+2]]for i in range(0,len(m),3)]
 for r in m:d=d.replace(r[1],r[0])
 d=eval(d)
 for k in d:
  if k['I']==g:g=k['O'];return g"""
    return sc

def OneBall(s):
    foundball=[]
    hits = []
    for k in s:
        if str(k['input']) not in foundball:
            foundball.append(str(k['input']))
            hits.append(k)
    return hits
    
f =[]
scores=[]
for n in range(1,401):
    t = load_examples(n)
    s = t['train'] + t['test'] + t['arc-gen']
    s = OneBall(s)
    s,m = theEagle(s)
    s = theScorecard(s,m)
    s = zip_src(bytes(s,'L1'))
    c = len(s)
    o=open('task' + str(n).zfill(3) + '.py','wb')
    o.write(s)
    o.close()
    if c<2500:
        print(n,c)
        f.append(n)
        scores.append(c)

print(len(f), len(f)*2500, sum([2500-x for x in scores]))
with ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for x in f:
        zipf.write('task' + str(x).zfill(3) + '.py')

In [ ]:
#https://www.kaggle.com/code/taylorsamarel/qwen2-5-32b-arc-local-score-32-solved-script
import zipfile, json, os, copy

def check(solution, task_num, valall=False):
    task_data = load_examples(task_num)
    #print(task_num, max(1, 2500 - len(solution.encode('utf-8'))))
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace: return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        examples_to_check = all_examples if valall else all_examples[:3]
        for example in examples_to_check:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except:
                return False
        return True
    except Exception as e:
        return False

In [ ]:
for task_num in f:
    try:
        solution = open('/kaggle/working/task' + str(task_num).zfill(3) + '.py','rb').read()
        if check(solution, task_num, valall=True):
            print(task_num, ":)")
        else: 
            print(task_num, ":L")
    except: pass

# Ｈ𝐀𝑷𝑷𝓎 🇰𝗮𝘨𝘨🇱𝖎Ｎɢ 💯